In [1]:
import pandas_plink as pp
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

#### Take a look at the gene_expression, gene_annotation files we have, we will subset gene expression data to find genes that are protein coding

In [4]:
# Path to the data folder
data_folder = '../data/'

In [6]:
gene_expression_file = data_folder + 'GD462.GeneQuantRPKM.50FN.samplename.resk10.txt'
gene_expression_data = pd.read_csv(gene_expression_file, delimiter='\t')

#slice the string to make the gene symbols consistent with gene_annotation data
gene_expression_data['TargetID'] = gene_expression_data['TargetID'].str.split('.').str[0]
gene_expression_data['Gene_Symbol'] = gene_expression_data['Gene_Symbol'].str.split('.').str[0]

In [7]:
gene_expression_data

,TargetID,Gene_Symbol,Chr,Coord,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
0,ENSG00000152931,ENSG00000152931,5,59783540,0.101858,0.078110,0.048981,0.118597,0.004035,0.010925,...,0.088601,0.240010,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
1,ENSG00000183696,ENSG00000183696,7,48128225,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,...,13.428205,6.094500,12.536000,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
2,ENSG00000139269,ENSG00000139269,12,57846106,1.199910,1.573572,0.521616,1.447225,3.565791,1.982681,...,3.225880,1.996067,2.854923,2.267343,1.331201,2.187895,1.004250,3.003316,1.984362,1.684954
3,ENSG00000169129,ENSG00000169129,10,116164515,0.831940,0.069778,0.931086,0.620941,1.660668,0.570481,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
4,ENSG00000134602,ENSG00000134602,X,131157293,27.646422,24.395572,16.445374,24.806650,25.113349,19.233988,...,25.079490,28.725528,24.450520,27.264069,26.912814,29.509210,26.462331,25.624009,25.707741,22.824957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23717,ENSG00000235472,ENSG00000235472,13,29172970,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,...,47.905182,33.224977,39.827675,27.096811,34.686342,37.386766,30.598840,33.516674,32.017940,38.341888
23718,ENSG00000114423,ENSG00000114423,3,105588396,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,...,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117
23719,ENSG00000243312,ENSG00000243312,4,87791344,1.112114,0.831797,0.253228,0.271568,0.486086,1.362640,...,1.168991,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.857330,0.703369
23720,ENSG00000257337,ENSG00000257337,12,53448222,3.826396,6.045798,2.593872,4.447169,5.294657,4.106823,...,6.524398,6.017795,3.460273,3.905032,5.023161,5.333027,5.694370,8.142939,5.622043,6.026476


In [8]:
gene_annotation_file = data_folder + 'gene_annot.txt'
gene_annotation_data = pd.read_csv(gene_annotation_file, delimiter='\t')

In [9]:
gene_annotation_data

,ID,CHR,START,STOP,SYM,TYPE
0,DDX11L1,1,11868,13052,ENSG00000223972,transcribed_unprocessed_pseudogene
1,OR4F5,1,69090,70008,ENSG00000186092,protein_coding
2,FAM87B,1,817370,819834,ENSG00000177757,lincRNA
3,LINC00115,1,826205,827522,ENSG00000225880,lincRNA
4,LINC01128,1,827607,859446,ENSG00000228794,processed_transcript
...,...,...,...,...,...,...
20495,ARSA,22,50622753,50628179,ENSG00000100299,protein_coding
20496,SHANK3,22,50674414,50733298,ENSG00000251322,protein_coding
20497,ACR,22,50738195,50745334,ENSG00000100312,protein_coding
20498,RPL23AP82,22,50756947,50801309,ENSG00000184319,transcribed_unprocessed_pseudogene


### Subset gene_expression data

In [12]:
# Filter for protein-coding genes
protein_coding_genes = gene_annotation_data[gene_annotation_data['TYPE'] == 'protein_coding']

# Get a list of protein-coding gene symbols
protein_coding_symbols = protein_coding_genes['SYM'].tolist()

# Subset gene_expression_data for protein-coding genes
protein_coding_expression_data = gene_expression_data[gene_expression_data['Gene_Symbol'].isin(protein_coding_symbols)]

In [13]:
protein_coding_expression_data

,TargetID,Gene_Symbol,Chr,Coord,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
1,ENSG00000183696,ENSG00000183696,7,48128225,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,...,13.428205,6.094500,12.536000,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
2,ENSG00000139269,ENSG00000139269,12,57846106,1.199910,1.573572,0.521616,1.447225,3.565791,1.982681,...,3.225880,1.996067,2.854923,2.267343,1.331201,2.187895,1.004250,3.003316,1.984362,1.684954
3,ENSG00000169129,ENSG00000169129,10,116164515,0.831940,0.069778,0.931086,0.620941,1.660668,0.570481,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
5,ENSG00000136237,ENSG00000136237,7,22396763,3.788503,2.050963,4.000313,3.271619,1.798216,1.516688,...,2.909393,1.921176,5.083873,2.866573,1.297788,2.888316,2.145022,3.557598,4.152063,1.216834
13,ENSG00000146072,ENSG00000146072,6,47277641,2.567374,2.030769,0.927560,1.351749,2.316378,0.931071,...,0.761671,0.797139,0.328339,0.156504,1.118230,0.240833,1.022178,0.393550,0.007050,0.642450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23712,ENSG00000087095,ENSG00000087095,17,26369182,5.490282,5.265422,3.457973,5.075550,5.571885,5.262668,...,6.678943,4.996638,5.125125,5.383470,5.206981,5.643264,5.290116,5.911672,5.249228,5.347071
23714,ENSG00000162144,ENSG00000162144,11,61129771,33.050457,23.159734,16.050400,20.848751,32.245357,26.842346,...,22.962417,26.718492,17.096197,36.692807,32.645718,26.245967,25.040414,33.586088,27.436847,24.236840
23715,ENSG00000129473,ENSG00000129473,14,23767999,5.960860,10.834208,4.606371,5.274462,6.453945,11.956208,...,7.947619,5.826466,6.691473,11.671064,9.606188,7.016099,6.305954,6.498428,4.919154,7.285334
23718,ENSG00000114423,ENSG00000114423,3,105588396,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,...,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117


## Subset SNPs within +-500 Kb windows, and then conduct eQTL analysis for each of the 22 chromosomes

#### Create a dataframe that will incorporate all results of 22 chromosomes, and create the function to read in genotype data

In [14]:
# Initialize an empty DataFrame to store results
all_results = pd.DataFrame()

# Function to read PLINK files for a specific chromosome
def read_plink_chromosome(chromosome_number):
    plink_folder = data_folder + 'LDREF/'
    plink_prefix = plink_folder + f'1000G.EUR.{chromosome_number}'
    return pp.read_plink(plink_prefix)

#### Chromosome 1

In [15]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '1']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 1] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(1)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()



Mapping files: 100%|██████████| 3/3 [00:00<00:00, 11.63it/s]


In [16]:
len(intersection_ids)

344

In [17]:
gene_annot

,ID,CHR,START,STOP,SYM,TYPE
0,DDX11L1,1,11868,13052,ENSG00000223972,transcribed_unprocessed_pseudogene
1,OR4F5,1,69090,70008,ENSG00000186092,protein_coding
2,FAM87B,1,817370,819834,ENSG00000177757,lincRNA
3,LINC00115,1,826205,827522,ENSG00000225880,lincRNA
4,LINC01128,1,827607,859446,ENSG00000228794,processed_transcript
...,...,...,...,...,...,...
2139,SH3BP5L,1,248810448,248826633,ENSG00000175137,protein_coding
2140,MIR3124,1,248826376,248826443,ENSG00000264500,miRNA
2141,ZNF672,1,248838209,248849517,ENSG00000171161,protein_coding
2142,ZNF692,1,248850005,248859085,ENSG00000171163,protein_coding


In [18]:
expression_data_filtered

,NA20581,HG00277,NA20799,HG00180,HG00264,HG00381,HG00372,NA20772,HG01791,HG00115,...,NA12718,HG00252,NA20504,NA20515,HG00326,NA07056,NA12778,HG00121,NA12413,NA12348
14,2.749185,2.728544,2.730144,4.359051,4.651351,3.189750,2.993184,3.879844,3.191631,1.488884,...,3.803506,5.094178,5.915144,3.918478,4.542814,3.282750,4.319513,3.722620,7.392088,5.495256
17,5.637124,5.055427,-1.095338,3.199341,6.612252,1.449575,6.634505,-1.951744,5.834815,1.847980,...,2.026301,1.578335,5.320268,0.809630,4.276337,13.203988,3.625861,3.861601,39.100838,13.060729
64,0.066064,0.283074,0.069232,0.130111,0.059283,-0.012758,0.035123,0.034455,0.103233,0.065562,...,0.158268,0.042356,0.104186,0.026417,0.082395,0.062630,0.084719,0.116357,0.031215,0.047240
72,0.543428,0.358011,1.359402,0.508256,0.663733,0.605640,0.743010,0.969807,0.512179,0.462170,...,0.357547,0.858662,0.615691,0.936637,0.592057,0.704004,0.766123,0.388523,0.424280,0.400706
74,0.032682,0.018164,0.029108,0.134324,0.181505,0.271835,-0.031909,0.102757,0.243646,0.014957,...,0.167848,0.133654,0.088081,0.146101,-0.054541,0.041426,0.781877,-0.051825,0.135623,1.917622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23600,1.900319,1.264244,0.986276,1.128541,1.217299,1.741634,1.417147,1.002235,0.941493,1.232987,...,0.481206,1.886135,1.306398,3.003186,0.496197,2.355953,0.724095,2.141281,0.715529,0.688207
23616,4.065664,4.830504,4.265805,6.868198,10.231141,4.733259,3.483288,10.232945,5.956079,5.700984,...,4.020621,6.203718,4.813290,4.219717,5.116473,5.283924,4.951325,5.478567,9.554028,7.775267
23622,4.450475,2.900151,2.927159,5.171821,4.101903,1.642093,3.191596,5.731440,5.218274,2.506862,...,1.957616,4.795253,5.222848,6.665778,1.460992,7.212469,3.656479,2.855420,2.144132,2.601503
23680,2.251077,0.534504,0.938240,0.309582,2.451371,1.659281,1.196890,1.644671,1.694738,1.613490,...,1.682486,1.130446,-0.496618,1.085239,-0.080288,0.916997,0.843959,0.463381,1.662580,-0.142445


In [23]:
#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

#### Chromosome 2

In [24]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '2']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 2] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(2)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00,  9.13it/s]


#### Chromosome 3

In [25]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '3']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 3] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(3)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 10.32it/s]


#### Chromosome 4

In [26]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '4']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 4] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(4)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00,  8.99it/s]


#### Chromosome 5

In [27]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '5']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 5] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(5)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 14.60it/s]


#### Chromosome 6

In [28]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '6']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 6] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(6)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00,  9.89it/s]


#### Chromosome 7

In [29]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '7']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 7] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(7)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 13.69it/s]


#### Chromosome 8

In [30]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '8']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 8] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(8)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 13.37it/s]


#### Chromosome 9

In [32]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '9']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 9] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(9)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 22.89it/s]


#### Chromosome 10

In [33]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '10']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 10] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(10)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 16.88it/s]


#### Chromosome 11

In [34]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '11']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 11] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(11)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 11.12it/s]


#### Chromosome 12

In [35]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '12']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 12] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(12)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 15.73it/s]


#### Chromosome 13

In [36]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '13']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 13] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(13)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 17.70it/s]


#### Chromosome 14

In [37]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '14']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 14] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(14)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 17.52it/s]


#### Chromosome 15

In [38]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '15']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 15] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(15)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 18.15it/s]


#### Chromosome 16

In [39]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '16']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 16] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(16)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 15.88it/s]


#### Chromosome 17

In [40]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '17']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 17] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(17)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 21.79it/s]


#### Chromosome 18

In [41]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '18']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 18] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(18)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 19.46it/s]


#### Chromosome 19

In [42]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '19']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 19] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(19)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 24.30it/s]


#### Chromosome 20

In [43]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '20']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 20] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(20)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 28.58it/s]


#### Chromosome 21

In [44]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '21']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 21] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(21)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 25.47it/s]


#### Chromosome 22

In [46]:
expression_data = protein_coding_expression_data[protein_coding_expression_data['Chr'] == '22']  
gene_annot = gene_annotation_data[gene_annotation_data['CHR'] == 22] 

# Read plink files
(bim, fam, bed) = read_plink_chromosome(22)

#Find common individuals
fam_ids = fam['fid'].tolist()
expression_ids = expression_data.columns.tolist()
intersection_ids = set(fam_ids).intersection(set(expression_ids))
intersection_ids_list = list(intersection_ids)

# Filter the expression data and genotype data to include only common individuals
expression_data_filtered = expression_data[intersection_ids_list]
bed_filtered = bed[:, fam[fam['fid'].isin(intersection_ids_list)].index].compute()

#Cis-eQTL Analysis
results = []
window_size = 500000  # 500 Kb window size
for gene_row in gene_annot.itertuples():
    gene_id = gene_row.ID
    gene_sym = gene_annot[gene_annot["ID"] == gene_id]["SYM"].iloc[0]
    gene_start = gene_row.START
    gene_end = gene_row.STOP

    # Filter SNPs within the window of the gene
    cis_snps = bim[(bim['pos'] > gene_start - window_size) & (bim['pos'] < gene_end + window_size)]

    if cis_snps.empty:
        continue
    gene_expression_df = expression_data_filtered[expression_data["TargetID"] == gene_sym]

    if gene_expression_df.empty:
        continue

    gene_expression = gene_expression_df.iloc[0]

    for snp_row in cis_snps.itertuples():
        snp_id = snp_row.snp
        snp = bed_filtered[snp_row.Index, :]

        if len(gene_expression) != len(snp):
            # Handle mismatch in length between gene expression and SNP data
            continue

        # Add a constant term to the independent variables to represent the intercept
        snp_with_intercept = sm.add_constant(snp)

        # Perform linear regression
        model = sm.OLS(gene_expression, snp_with_intercept)
        result = model.fit()

        # Check if there are enough parameters in the results
        if len(result.params) < 2:
            continue

        pvalue = result.pvalues[1]
        beta = result.params[1]
        std_err = result.bse[1]

        results.append({
            'gene_id': gene_id,
            'gene_symbol': gene_sym,
            'snp_id': snp_id,
            'pvalue': pvalue,
            'OR': beta,
            'SE': std_err,
            'snp_position': snp_row.pos
        })

all_results = pd.concat([all_results] + [pd.DataFrame([res]) for res in results], ignore_index=True)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 23.06it/s]


### Total results of all 22 chromosomes

In [47]:
all_results

,gene_id,gene_symbol,snp_id,pvalue,OR,SE,snp_position
0,SAMD11,ENSG00000187634,rs3094315,0.298244,0.094348,0.090563,752566
1,SAMD11,ENSG00000187634,rs3131972,0.276524,0.098526,0.090400,752721
2,SAMD11,ENSG00000187634,rs3131969,0.538234,0.058996,0.095756,754182
3,SAMD11,ENSG00000187634,rs1048488,0.317524,0.089695,0.089604,760912
4,SAMD11,ENSG00000187634,rs3115850,0.302921,0.092290,0.089450,761147
...,...,...,...,...,...,...,...
6183028,RABL2B,ENSG00000079974,rs13056621,0.850607,-0.051107,0.271142,51181759
6183029,RABL2B,ENSG00000079974,rs3865766,0.322009,-0.178194,0.179672,51186228
6183030,RABL2B,ENSG00000079974,rs3888396,0.511601,0.164925,0.251014,51211392
6183031,RABL2B,ENSG00000079974,rs2238837,0.338360,-0.184742,0.192690,51212875


### The following codes are used to store the all_results dataframe to a local spreadsheet, so that we don't have to run the above analysis everytime we restart the notebook (jump to "Discovering the significant SNPs" if you don't want to save it to local folder)

In [49]:
max_rows = 1048576  # Excel's limit
num_sections = len(all_results) // max_rows + 1

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter('large_file.xlsx', engine='xlsxwriter') as writer:
    for section in range(num_sections):
        start_row = max_rows * section
        end_row = start_row + max_rows
        # Write each section of the DataFrame to a different sheet
        all_results.iloc[start_row:end_row].to_excel(writer, sheet_name=f'Project1_eQTL_results{section + 1}', index=False)

In [9]:
file_pattern = '*.xlsx'  # Assumes files have .xlsx extension

# Create a list of all Excel files in the directory
excel_files = glob.glob(file_pattern)

# List of sheet names to read
sheet_names = ['Project1_eQTL_results1', 'Project1_eQTL_results2', 'Project1_eQTL_results3', 
               'Project1_eQTL_results4', 'Project1_eQTL_results5', 'Project1_eQTL_results6']

In [10]:
excel_files

['large_file.xlsx']

In [11]:
# Read each sheet in each Excel file and store in a list
dfs = []
for file in excel_files:
    for sheet in sheet_names:
        df = pd.read_excel(file, sheet_name=sheet)
        dfs.append(df)
saved_results = pd.concat(dfs, ignore_index=True)

In [12]:
saved_results

,gene_id,gene_symbol,snp_id,pvalue,OR,SE,snp_position
0,SAMD11,ENSG00000187634,rs3094315,0.298244,0.094348,0.090563,752566
1,SAMD11,ENSG00000187634,rs3131972,0.276524,0.098526,0.090400,752721
2,SAMD11,ENSG00000187634,rs3131969,0.538234,0.058996,0.095756,754182
3,SAMD11,ENSG00000187634,rs1048488,0.317524,0.089695,0.089604,760912
4,SAMD11,ENSG00000187634,rs3115850,0.302921,0.092290,0.089450,761147
...,...,...,...,...,...,...,...
6183023,RABL2B,ENSG00000079974,rs13056621,0.850607,-0.051107,0.271142,51181759
6183024,RABL2B,ENSG00000079974,rs3865766,0.322009,-0.178194,0.179672,51186228
6183025,RABL2B,ENSG00000079974,rs3888396,0.511601,0.164925,0.251014,51211392
6183026,RABL2B,ENSG00000079974,rs2238837,0.338360,-0.184742,0.192690,51212875


### Discovering the significant SNPs

#### Filter with P-Value of 0.01

In [13]:
df = saved_results[saved_results["pvalue"] < 0.01]
df

,gene_id,gene_symbol,snp_id,pvalue,OR,SE,snp_position
95,SAMD11,ENSG00000187634,rs4970421,0.001869,-0.510259,0.162775,1108637
301,NOC2L,ENSG00000188976,rs9729550,0.003789,-0.841076,0.288513,1135242
342,NOC2L,ENSG00000188976,rs604618,0.007438,-1.257599,0.467054,1232319
343,NOC2L,ENSG00000188976,rs1739855,0.003529,-1.277167,0.434732,1233941
344,NOC2L,ENSG00000188976,rs2477782,0.002817,-1.300435,0.432199,1237036
...,...,...,...,...,...,...,...
6182741,RABL2B,ENSG00000079974,rs9628283,0.000096,-2.913991,0.738166,50540766
6182918,RABL2B,ENSG00000079974,rs140524,0.009310,0.657288,0.251325,50960682
6182932,RABL2B,ENSG00000079974,rs6520149,0.001168,1.964061,0.599877,50990669
6182933,RABL2B,ENSG00000079974,rs6009921,0.001168,1.964061,0.599877,50991205


#### Sort the dataframe after filtration by ascending p-value and descending OR (effect sizes)

In [14]:
df['abs_OR'] = df['OR'].abs()

# Sort the DataFrame first by pvalue in ascending order, then by abs_OR in descending order
sorted_df = df.sort_values(by=['pvalue', 'abs_OR'], ascending=[True, False])

sorted_df

C:\Users\13269\AppData\Local\Temp\ipykernel_14020\1554700442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['abs_OR'] = df['OR'].abs()


,gene_id,gene_symbol,snp_id,pvalue,OR,SE,snp_position,abs_OR
203400,STIL,ENSG00000123473,rs1105456,2.586935e-23,-25.804894,2.406922,47269984,25.804894
2363128,PPP1R17,ENSG00000106341,rs16875607,1.683112e-21,-0.573896,0.056271,31657503,0.573896
2363134,PPP1R17,ENSG00000106341,rs10262468,1.683112e-21,-0.573896,0.056271,31667929,0.573896
753984,CCDC85A,ENSG00000055813,rs12622380,4.921446e-20,-5.514587,0.564650,55702485,5.514587
4120391,CCNA1,ENSG00000133101,rs9546280,9.198354e-19,-3.228637,0.344107,36638466,3.228637
...,...,...,...,...,...,...,...,...
75974,MINOS1,ENSG00000173436,rs12565770,9.999321e-03,-3.986838,1.539138,19555060,3.986838
2362430,GHRHR,ENSG00000106128,rs17159410,9.999359e-03,-0.023909,0.009230,30779725,0.023909
3271155,C10orf12,ENSG00000155640,rs943544,9.999718e-03,0.053292,0.020574,97148731,0.053292
5728081,ZNF444,ENSG00000167685,rs2278281,9.999788e-03,0.933497,0.360383,55668197,0.933497


#### The first row is the most significant SNP.